In [0]:
import numpy as np
import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 734 (delta 67), reused 97 (delta 36), pack-reused 600
Receiving objects: 100% (734/734), 1.28 MiB | 4.81 MiB/s, done.
Resolving deltas: 100% (393/393), done.


In [3]:
BATCH_SIZE = 512 #@param ["512", "256", "128"] {type:"raw"}
MOMENTUM = 0.9 #@param ["0.9", "0.95", "0.975"] {type:"raw"}
WEIGHT_DECAY = 0.000125 #@param ["0.000125", "0.00025", "0.0005"] {type:"raw"}
LEARNING_RATE = 0.4 #@param ["0.4", "0.2", "0.1"] {type:"raw"}
EPOCHS = 24 #@param {type:"slider", min:0, max:100, step:1}
WARMUP = 5 #@param {type:"slider", min:0, max:24, step:1}
BUCKET = 'gs://gs_colab'
PROJECT = 'cifar10'

In [4]:
fw.colab_utils.setup_gcs()

In [5]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
n_train, n_test = X_train.shape[0], X_test.shape[0]
img_size = X_train.shape[1]
n_classes = y_train.max() + 1

In [7]:
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_std = np.std(X_train, axis=(0,1,2))
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [8]:
train_fn = os.path.join(data_dir, "train.tfrec")
test_fn = os.path.join(data_dir, "test.tfrec")

fw.io.numpy_tfrecord(train_fn, X_train, y_train)
fw.io.numpy_tfrecord(test_fn, X_test, y_test)

In [9]:
def parser_train(tfexample):
  x, y = fw.io.tfexample_numpy_image_parser(tfexample, img_size, img_size)
  x = fw.transform.ramdom_pad_crop(x, 4)
  x = fw.transform.random_flip(x)
  x = fw.transform.cutout(x, 8, 8)
  return x, y

parser_test = lambda x: fw.io.tfexample_numpy_image_parser(x, img_size, img_size)

In [10]:
train_input_func = lambda params: fw.io.tfrecord_ds(train_fn, parser_train, batch_size=params['batch_size'], training=True)
eval_input_func = lambda params: fw.io.tfrecord_ds(test_fn, parser_test, batch_size=params['batch_size'], training=False)

In [11]:
def build_nn(c=64, weight=0.125):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvResBlk(c*2, res_convs=2, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvResBlk(c*8, res_convs=2, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(fw.layers.Classifier(n_classes, kernel_initializer=fw.layers.init_pytorch, weight=0.125))
  return model

In [12]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS

In [13]:
lr_func = fw.train.triangular_lr(LEARNING_RATE/BATCH_SIZE, total_steps, warmup_steps=WARMUP*steps_per_epoch)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
opt_func = fw.train.sgd_optimizer(lr_func, mom=MOMENTUM, wd=WEIGHT_DECAY*BATCH_SIZE)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func, reduction=tf.losses.Reduction.SUM)

In [15]:
est = fw.tpuest.get_tpu_estimator(n_train, n_test, model_func, work_dir, trn_bs=BATCH_SIZE)
est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/cifar10/2019-04-19-14:03:45', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.32.164.234:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc8d0a94a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.32.164.234:8470', '_evaluation_master': 'grpc://10.32.164.234:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=97, num_shards=Non

In [16]:
result = est.evaluate(eval_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-19T14:04:48Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/cifar10/2019-04-19-14:03:45/model.ckpt-2328
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 8 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Stop infeed thread controller
INFO:tensorflow:Shutting down InfeedController thread.
INFO:tensorflow:Inf

In [17]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 94.08%, loss= 202.99.


In [18]:
fw.io.create_clean_dir(work_dir)